In [ ]:
import os, sys
import numpy as np
import torch
torch.set_default_dtype(torch.float64)
import abakit.registration as reg

In [ ]:
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.SqlController import SqlController
from utilities.utilities_cvat_neuroglancer import get_structure_number
atlas_name = 'atlas'
sqlController = SqlController(atlas_name)

In [ ]:
def align_point_sets(src, dst, with_scaling=True):
    torch.set_default_dtype(torch.float64)

    registration = reg.LandmarkRegistration(
        dst,
        src
    )

    transform = reg.LandmarkAffineTransform()
    transform.init_guess(
        dst,
        src
    )
    registration.set_transform(transform)

    #registration.set_similarity_metric(reg.MSESimilarity())
    registration.set_optimizer_class(torch.optim.Adam)

    registration.run(max_iteration=5000)
    
    r = transform.get_linear_transform_matrix()
    t = transform.get_translation()
    
    return r.T, t.reshape([1, -1]).T

In [ ]:
src = sqlController.get_centers_dict('DK52', person_id=2, input_type_id=2)
dst = sqlController.get_centers_dict('Atlas', person_id=1, input_type_id=4)
structures = sorted(src.keys())
common_keys = src.keys() & dst.keys()
fixed_landmark = np.array([dst[s] for s in structures if s in common_keys])
moving_landmark = np.array([src[s] for s in structures if s in common_keys])

In [ ]:
registration = reg.LandmarkRegistration(
    fixed_landmark,
    moving_landmark
)

transform = reg.LandmarkAffineTransform()
transform.init_guess(
    fixed_landmark,
    moving_landmark
)
registration.set_transform(transform)

registration.set_similarity_metric(reg.MSESimilarity())
registration.set_optimizer_class(torch.optim.Adam)

registration.run(max_iteration=5000)

In [ ]:
R, t = align_point_sets(moving_landmark, fixed_landmark)